<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/limpieza/limpieza_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descripción de la documentación.

En este documento se revisan las bases de datos para limpiar la información de datos nulos, columnas con información considerara innecesaria para la API o el modelo de Machine Learning, así como de dejar pre-procesada parte de la data para facilitar el trabajo de la documentación en posteriores iteraciones.

# Descarga de bibliotecas a utilizar

In [ ]:
# Se importan las librerías a usar.
import pandas as pd
import gzip
import json
import re
from pandas import json_normalize

In [ ]:
# Ruta de los archivos a utilizar
ruta_items = "/content/items_corregido.json.gz"
ruta_reviews = "/content/reviews_corregido.json.gz"
ruta_games = "/content/steam_games.json.gz"

In [ ]:
# se crean los dataframes con los que se trabajará.
items = pd.read_json(ruta_items,compression="gzip")

In [ ]:
reviews = pd.read_json(ruta_reviews,compression = "gzip")

In [ ]:
games = pd.DataFrame(pd.read_json(ruta_games,compression="gzip",lines=True))

In [ ]:
# Se crea una función para guardar los nuevos archivos corregidos

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Limpieza games

In [ ]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [ ]:
# Se eliminan las filas cuyos datos sean todos nulos (ya que no nos proporcionan información sobre el juego).
games.dropna(how = "all", inplace = True)
# Se procede a eliminar las columnas con mayor número de datos incompletos o que se repiten, así como aquellos cuya información no resulta útil para la formulación de las consultas.
games.drop(["title","url","reviews_url","price","early_access","publisher","genres"], axis = 1, inplace = True)


In [ ]:
# Se revisa la base de datos obtenida a partir de la limpieza y se cambian los nombres de las columnas.
games.rename(columns={"app_name":"name","tags":"genre","id":"id_game"},inplace=True)
games

,name,release_date,genre,specs,id_game,developer
88310,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140.0,Kotoshiro
88311,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980.0,Secret Level SRL
88312,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",670290.0,Poolians.com
88313,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],767400.0,彼岸领域
88314,Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",773570.0,None
...,...,...,...,...,...,...
120440,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",773640.0,"Nikita ""Ghost_RUS"""
120441,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",733530.0,Sacada
120442,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",610660.0,Laush Dmitriy Sergeevich
120443,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",658870.0,"xropi,stev3ns"


In [ ]:
# Se busca la información obtenida hasta el momento.
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         32135 non-null  int64  
 1   name          32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   genre         31972 non-null  object 
 4   specs         31465 non-null  object 
 5   id_game       32133 non-null  float64
 6   developer     28836 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


In [ ]:
# Se eliminan las filas que no tengan un valor para el ID del juego ("id_game")
games = games.dropna(subset=["id_game"])

In [ ]:
# Se transforma el valor de "id_game" para que sea un entero.
games["id_game"] = games["id_game"].astype(int)

<ipython-input-37-c51d55476d29>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["id_game"] = games["id_game"].astype(int)


In [ ]:
# Se revisa que el cambio haya sido exitoso.
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32133 entries, 88310 to 120444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          32132 non-null  object
 1   release_date  30067 non-null  object
 2   genre         31971 non-null  object
 3   specs         31464 non-null  object
 4   id_game       32133 non-null  int64 
 5   developer     28835 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.7+ MB


In [ ]:
#Se reinician los índex
games.reset_index(inplace=True)

In [ ]:
# Se revisa la información obtenida.
games

,index,name,release_date,genre,specs,id_game,developer
0,88310,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140,Kotoshiro
1,88311,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980,Secret Level SRL
2,88312,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",670290,Poolians.com
3,88313,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],767400,彼岸领域
4,88314,Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",773570,None
...,...,...,...,...,...,...,...
32128,120440,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",773640,"Nikita ""Ghost_RUS"""
32129,120441,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",733530,Sacada
32130,120442,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",610660,Laush Dmitriy Sergeevich
32131,120443,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",658870,"xropi,stev3ns"


In [ ]:
# Se crea un nuevo dataframe para los géneros y para las especificaciones.

games_genres = games[["id_game","genre"]]
games_specs = games[["id_game","specs"]]

# Se eliminan estas columnas del primer data set (ya que ya no son necesarias) así como la columna index.
games.drop(columns=["genre","specs","index"],inplace = True)

<ipython-input-41-7e5067060ab4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.drop(columns=["genre","specs","index"],inplace = True)


In [ ]:
# Se desagregan los datos del género para cada uno de los juegos.
games_genres = games_genres.explode(column=["genre"],ignore_index=True)
games_specs = games_specs.explode(column="specs",ignore_index=True)

In [ ]:
# Se guardan los archivos corregidos en nuevos documentos.
guardar_archivo(games,"/content/games_clean.json.gz")
guardar_archivo(games_genres,"/content/genres_games.json.gz")
guardar_archivo(games_specs,"/content/specs_games.json.gz")

# Limpieza Reviews

In [ ]:
# Se visualizan los datos del data frame.
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25761,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25762,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25763,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25764,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [ ]:
# Se elimina la columna "user_url", ya que no será usada.
reviews.drop(columns="user_url",inplace=True)

In [ ]:
# Se desagregan cada una de las reviews para cada usuario.
reviews = reviews.explode(column="reviews",ignore_index=True)

In [ ]:
reviews

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
59187,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59188,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59189,LydiaMorley,"{'funny': '1 person found this review funny', ..."
59190,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [ ]:
# Con el propósito de separar cada una de las reviews, se normaliza la columna "reviews" la cual se encuentra en formato JSON en un DataFrame nuevo.
x = json_normalize(reviews["reviews"])
# Se combina el DataFrame normalizado con el DataFrame original.
reviews = pd.concat([reviews, x], axis=1).drop('reviews', axis=1)
reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59187,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59188,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59189,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59190,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [ ]:
# Se eliminan las columnas que ya no son necesarias o útiles para las consultas que se realizarán.
reviews.drop(columns=["funny","helpful"],inplace=True)

In [ ]:
# Se reemplazan los valores no deseados (espacios en blanco) en "last_edited"
reviews["last_edited"] = reviews["last_edited"].replace(r"^\s*$", pd.NA, regex=True)

In [ ]:
# Se coincidera que los valores de "last_edited" son los valores de la review más reciente (y por tanto, al año al que pertenecen).
# De esta forma, se decide dejar estos datos, reemplazando aquellos donde se tiene un dato nulo por el valor de la columna "posted"
reviews["last_edited"].fillna(reviews["posted"],inplace=True)

In [ ]:
# Se elimina la columna "posted" por redundancia con la columna "last_edited"
reviews.drop(columns="posted",inplace=True)

In [ ]:
# Se separa la columna "last_edited" para poder obtener la fecha en una columna separada.
reviews[["edit_status","edit_date"]] = reviews["last_edited"].str.split(" ",n= 1,expand=True)
# Se elimina la columna "last_edited" y la columna "edit_status" por redundancia y/o por resultar innecesaria.
reviews.drop(columns=["last_edited","edit_status"],inplace = True)

In [ ]:
# Se transforman a formato fecha los valores en "edit_date", dejando aquellos datos no válidos como NaT.
reviews["edit_date"] = pd.to_datetime(reviews["edit_date"],errors="coerce")
# Se cambia el nombre de "edit_date" a "review_date"
reviews.rename(columns={"edit_date":"review_date"},inplace=True)

In [ ]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
59187,76561198312638244,70,True,a must have classic from steam definitely wort...,NaT
59188,76561198312638244,362890,True,this game is a perfect remake of the original ...,NaT
59189,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,NaT
59190,LydiaMorley,730,True,:D,NaT


In [ ]:
# Se decide eliminar todos los valores cuya fecha no corresponde a un formato de fecha válido, ya que no se puede obtener el año de la reseña.
reviews.dropna(inplace=True)

In [ ]:
# Se restablecen los indexes.
reviews = reviews.reset_index(drop=True)

In [ ]:
# Se hace una visualización de la data.
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [ ]:
# Se reemplazan las reviews vacías por un mensaje que diga "No review"
reviews["review"].replace([""," "],"No review",inplace = True)

In [ ]:
# Se aseguran que todos los datos en "review" sean string.
reviews["review"] = reviews["review"].astype(str)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43976 entries, 0 to 43975
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      43976 non-null  object        
 1   item_id      43976 non-null  object        
 2   recommend    43976 non-null  object        
 3   review       43976 non-null  object        
 4   review_date  43976 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 1.7+ MB


In [ ]:
# Se guarda la data transformada en un nuevo archivo.
guardar_archivo(reviews,"/content/reviews_limpia.json.gz")

In [ ]:
# Revisión de la correcta descompresión
r_limpio = pd.read_json("/content/reviews_limpia.json.gz",compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
r_limpio

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


# Limpieza Items

In [ ]:
# Se visualizan los datos del dataset.
items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88298,76561198323066619,22,76561198323066624,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88299,76561198326700687,177,76561198326700688,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88300,XxLaughingJackClown77xX,0,76561198328759264,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88301,76561198329548331,7,76561198329548336,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [ ]:
# Se eliminan las columnas que no proporcionan información necesaria para nuestro proyecto.
items.drop(columns=["steam_id","user_url","items_count"],inplace=True)

In [ ]:
# Se desagregan los vídeojuegos por usuario.
items = items.explode("items",ignore_index=True)

In [ ]:
# Se desagregan los datos en la columna items a través de la creación de un nuevo dataframe normalizando los datos.
o = json_normalize(items["items"])
# Se combina el DataFrame normalizado con el DataFrame original.
items = pd.concat([items, o], axis=1).drop("items", axis=1)
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5159891,76561198329548331,373330,All Is Dust,0.0,0.0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5159893,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0
5159894,76561198329548331,519140,Minds Eyes,3.0,3.0


In [ ]:
# Se eliminan las filas con datos nulos
items.dropna(inplace=True)

In [ ]:
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5159890,76561198329548331,346330,BrainBread 2,0.0,0.0
5159891,76561198329548331,373330,All Is Dust,0.0,0.0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5159893,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


In [ ]:
# Se guardan los datos en un nuevo archivo.
guardar_archivo(items,"/content/items_limpia.json.gz")